<style>div.title-slide {    width: 100%;    display: flex;    flex-direction: row;            /* default value; can be omitted */    flex-wrap: nowrap;              /* default value; can be omitted */    justify-content: space-between;}</style><div class="title-slide">
<span style="float:left;">Licence CC BY-NC-ND</span>
<span></span>
<span></span>
</div>

# WARNING

### This notebook is currently broken

As of beginning of March 2018, this won't work in a notebook; we are exactly in the scope of this issue https://github.com/jupyter/notebook/issues/3397 which has to do with tornado 5.0's loop having been unified with asyncio's.

# Simplest possible ssh scripts

In this notebook we will just run the simplest possible ssh scripts to the R2lab infrastructure:

* first simply to the gateway - you need a valid slice name for that to work
  * as an aside we also run an X11 command, so that we see how to enable X11 tunnelling.
* second on a node that we reach through the gateway

This cell for convenience only, for when the underlying libraries are being modified live.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from asynciojobs import Scheduler
from apssh import SshNode, SshJob, Run

### Single leg ssh connection : on `faraday` itself

In [ ]:
slicename = 'inria_r2lab.tutorial'

gw = SshNode('faraday.inria.fr', username=slicename)

##### single leg simple command

We always need at least one scheduler, this is our event loop:

In [ ]:
s = Scheduler()
print(s)

For this simplest possible scenario we need one `SshJob` instance; it needs to know
* on what node it will run
* what commands need to run
* optionnally in which scheduler - although it can also be added later on

In [ ]:
j = SshJob(node=gw,
           command='hostname',
           scheduler = s)
print(s)

The scheduler will run all its jobs until complete:

In [ ]:
s.run()

In [ ]:
print(s)

##### single leg, command with X11

If the command needs X11 forwarding, we need to enable that feature. Before we can do that,  note that:

```
j = SshJob(
    node=gw,
    command='hostname',
    scheduler = s)
```

can also be written, it is *exactly* equivalent, as

```
j = SshJob(
    node=gw,
    command=Run('hostname'),
    scheduler = s)
```

or even:
```
j = SshJob(
    node=gw,
    command=[ Run('hostname') ],
    scheduler = s)
```

Replacing a simple string like `"hostname"` with an instance of `Run` allows to define more advanced settings, like `X11=true` that we need here:

In [ ]:
s = Scheduler()
jx = SshJob(node=gw, scheduler=s, command=Run('xterm', x11=True))

In [ ]:
s.run()

### 2 legs - join a node through the gateway

**Note**: The following **assumes that `fit06` has been turned on**.

In [ ]:
node = SshNode("fit06", username="root", gateway=gw)

##### 2-leg ssh, simple command

In [ ]:
s = Scheduler()
j2 = SshJob(node=node,
            command='hostname',
            scheduler=s)

In [ ]:
s.run()

##### 2-leg ssh, X11 command

In [ ]:
s = Scheduler()
j2 = SshJob(node=node,
            command=Run('xterm', x11=True),
            scheduler=s)

In [ ]:
s.run()

### Using bash-written convenience tools on faraday

In [ ]:
from apssh import RunString

#####  Load `/etc/profile.d/faraday.sh`

Here's a trick that will let you use convenience shell tools as in a terminal.

You just need to explicitly load the goodies like this:

In [ ]:
my_script = """#!/bin/bash

# explicitly load faraday utilities
source /etc/profile.d/faraday.sh

# now we can use all convenience tools
# including selection-based ones

n 1-5
rstatus
"""

In [ ]:
s = Scheduler()
j = SshJob(node=gw,
           command=RunString(my_script,
                            remote_name = 'foo'),
           scheduler = s)
s.graph()

In [ ]:
s.run()

### Using bash-written convenience tools on nodes

With our images, nodes-oriented convenience tools are located in 

    /etc/profile.d/nodes.sh
    
and they can be used the same way as above.

# Embedding the testbed map in a notebook

In [ ]:
from IPython.display import IFrame
IFrame('https://r2lab.inria.fr/iframe.md', height=400, width='100%')